# Title
[]()

In [13]:
import requests
import json
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
import pickle
import os

In [14]:
with open("credentials.json") as f:
    access_token = json.load(f)['access_token']
user_id = os.environ['fb_user_id']

# Get Posts

In [2]:
def get_user_post(user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    user_id = str(user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url = f'{url_root}{user_id}/posts?access_token={access_token}'
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of posts:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

# SH 2023-01-16 16:58 Need to update this so that:
    # The URL of the last request is returned in case you want to request posts further back
    # If it is the last page, it won't just keep making request with the same endpoint

In [ ]:
df_5p, response_json_5p = get_user_post(user_id, access_token)
df_5p

In [ ]:
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
filename='my_fb_posts_5page_2023-01-12'
save_csv(df_5p,filename,csv_path)
savepickle(response_json_5p,filename,'sav',json_path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/my_fb_posts_5page_2023-01-12.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/my_fb_posts_5page_2023-01-12.sav


In [15]:
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
df_5p = load_csv('my_fb_posts_5page_2023-01-12.csv', csv_path, column1_as_index=True)
df_5p

Dataframe shape:  (119, 3)


,created_time,message,id
0,2023-01-15T20:21:48+0000,Too good to share. Too bad there's none on exe...,10104327314119821_10104331500165951
1,2023-01-13T19:50:47+0000,It’s been 1 month since I finished my data sci...,10104327314119821_10104329950905681
2,2023-01-11T03:17:40+0000,There was a time when my only exercise was run...,10104327314119821_10104327972240941
3,2023-01-09T21:30:18+0000,"For the US, ""Wearable technology (#1), strengt...",10104327314119821_10104326999794731
4,2023-01-09T17:06:33+0000,Excited to see developments in wearable tech!\...,10104327314119821_10104326849426071
...,...,...,...
18,2021-12-21T23:40:00+0000,Why the scale goes crazy during the holidays: ...,10104327314119821_10103960734697921
19,2021-12-17T05:59:42+0000,Feels like I am at a plateau with hand balanci...,10104327314119821_10103957435309921
20,2021-12-14T19:36:06+0000,New toy,10104327314119821_10103955698231041
21,2021-12-05T06:57:42+0000,NaN,10104327314119821_10103949690400781


In [ ]:
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
filename='my_fb_posts_5page_2023-01-12.sav'
response_json_5p = loadpickle(filename, json_path)
response_json_5p

## process_timestamp

In [17]:
import re
def process_timestamp(df):
    """
    Convert dates in the json-derived dataframe into different formats.
    """
    df.reset_index(drop=True, inplace=True)
    regex_date = r'.+T'
    df2 = pd.DataFrame()
    df2['timestamp'] = pd.to_datetime(df['created_time'])
    df2['text'] = df['message']
    df2['date'] = df2['timestamp'].dt.date
    df2['year'] = df2['timestamp'].dt.year
    df2['month'] = df2['timestamp'].dt.month
    df2['day_of_week'] = df2['timestamp'].dt.dayofweek
    df2['time'] = df2['timestamp'].dt.time
    df2['hour'] = df2['timestamp'].dt.hour
    return df2

process_timestamp(df_5p.head(3).copy())

,timestamp,text,date,year,month,day_of_week,time,hour
0,2023-01-15 20:21:48+00:00,Too good to share. Too bad there's none on exe...,2023-01-15,2023,1,6,20:21:48,20
1,2023-01-13 19:50:47+00:00,It’s been 1 month since I finished my data sci...,2023-01-13,2023,1,4,19:50:47,19
2,2023-01-11 03:17:40+00:00,There was a time when my only exercise was run...,2023-01-11,2023,1,2,03:17:40,3


In [18]:
process_timestamp(df_5p.head(3).copy()).dtypes

timestamp      datetime64[ns, UTC]
text                        object
date                        object
year                         int64
month                        int64
day_of_week                  int64
time                        object
hour                         int64
dtype: object

# Get Photos

## `get_user_photos`

In [ ]:
def get_user_photos(user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    user_id = str(user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url = f'{url_root}{user_id}/photos?type=uploaded&access_token={access_token}'
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of photos:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict
# SH 2023-01-16 16:58 Need to update this so that:
    # The URL of the last request is returned in case you want to request posts further back
    # If it is the last page, it won't just keep making request with the same endpoint

In [ ]:
df_photos, response_json_photos = get_user_photos(user_id, access_token, pages=2)

Response status code:  200


KeyboardInterrupt: 

In [ ]:
df_photos.head(10)

,created_time,name,id
0,2023-01-13T19:50:47+0000,It’s been 1 month since I finished my data sci...,10104329950835821
1,2023-01-07T21:36:58+0000,NaN,10104325301812501
2,2023-01-07T21:36:58+0000,NaN,10104325301787551
3,2023-01-07T21:36:58+0000,NaN,10104325301737651
4,2023-01-07T21:36:58+0000,NaN,10104325301687751
5,2023-01-06T19:39:51+0000,What's your favourite app or wearable that hel...,10104324466511451
6,2023-01-03T20:34:34+0000,Gamify your goals\n\nDuolingo was one of the p...,10104322209175171
7,2022-12-30T02:06:59+0000,NaN,10104318386994851
8,2022-12-30T02:06:59+0000,NaN,10104318386959921
9,2022-12-27T07:56:46+0000,While my podcast feed is mostly filled with po...,10104316147467881


In [ ]:
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
filename='my_fb_photos_2023-01-14'
save_csv(df_photos,filename,csv_path)
savepickle(response_json_photos,filename,'sav',json_path)

File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/my_fb_photos_2023-01-14.csv
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/my_fb_photos_2023-01-14.sav


In [ ]:
def save_response(df, response_json, filename,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """
    Save the data frame and json_response from the Facebook API request.
    """
    save_csv(df,filename,csv_path)
    savepickle(response_json,filename,'sav',json_path)
    


# Pulling AM's Posts

In [53]:
with open("credentials.json") as f:
    credentials = json.load(f)

access_token = credentials['am_access_token'] # SH 2023-01-25 23:59 long lived access token
am_user_id = credentials['am_user_id'] 
df, response_json = get_user_post(am_user_id, access_token, pages=1)

Response status code:  200
Number of posts: 24


In [52]:
with open("credentials.json") as f:
    credentials = json.load(f)

access_token = credentials['am2_access_token'] # SH 2023-01-25 23:59 short-lived access token
am_user_id = credentials['am_user_id']
df, response_json = get_user_post(am_user_id, access_token, pages=1)

Response status code:  400


KeyError: "Key 'data' not found. If specifying a record_path, all elements of data should have the path."

In [ ]:
df_50p, response_json_50p = get_user_post(am_user_id, access_token, pages=50, 
    filename='AM_fb_posts_50page_2023-01-16')

Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
Response status code:  200
R

In [20]:
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
df_50p = load_csv('AM_fb_posts_50page_2023-01-16.csv', csv_path, column1_as_index=True)

Dataframe shape:  (53, 3)


In [280]:
df_50p.shape

(53, 3)

In [ ]:
df_50p

,created_time,message,id
0,2023-01-04T15:06:57+0000,Charity event in Roscommon - Saturday 7th Janu...,545505737609234_537318481761293
1,2022-11-29T18:55:49+0000,Nutrition and Training Workshop 💪\n\nMy ethos ...,545505737609234_509802061179602
2,2022-10-04T22:39:50+0000,‘Why your metabolism is not broken’\n\nReally ...,545505737609234_467470882079387
3,2022-09-18T20:34:20+0000,Anyone else on TikTok? 🎯,545505737609234_455027709990371
4,2022-09-18T06:19:36+0000,Completed it mate ✅,545505737609234_454597556700053
5,2022-09-15T19:20:59+0000,Anyone doing Blackmores - marathon/half or 10km?,545505737609234_452894803536995
6,2022-08-23T08:49:02+0000,We had the most stunning time in Fiji 🇫🇯\n\nHa...,545505737609234_436768241816318
7,2022-08-16T02:04:54+0000,"If you can’t beat ‘em, join em 🍸",545505737609234_431993685627107
8,2022-08-14T23:22:47+0000,Bingo Loco 🫶,545505737609234_431270869032722
9,2022-08-10T21:13:31+0000,First night out out in awhile 🫶,545505737609234_428638645962611


In [2]:
def get_user_post(user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    user_id = str(user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url = f'{url_root}{user_id}/posts?access_token={access_token}'
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Request URL:', url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of posts:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

In [ ]:
df_photos_am, response_json_photos_am = get_user_photos(am_user_id, access_token, pages=50,
    filename='AM_fb_photos_50page_2023-01-16')

In [ ]:
df_photos_am

,created_time,name,id
0,2023-01-04T15:06:57+0000,Charity event in Roscommon - Saturday 7th Janu...,537318171761324
1,2022-11-29T18:55:49+0000,Nutrition and Training Workshop 💪\n\nMy ethos ...,509802031179605
2,2022-10-04T22:39:50+0000,‘Why your metabolism is not broken’\n\nReally ...,467470492079426
3,2022-09-18T06:19:36+0000,Completed it mate ✅,454597496700059
4,2022-08-23T08:49:02+0000,We had the most stunning time in Fiji 🇫🇯\n\nHa...,436768148482994
...,...,...,...
1,2021-03-12T00:33:05+0000,NaN,109633777863101
2,2021-03-12T00:25:31+0000,NaN,109630624530083
0,2021-04-27T04:45:39+0000,Snack Options ✅\n.\n.\nA question I get so fre...,126493682843777
1,2021-03-12T00:33:05+0000,NaN,109633777863101


In [ ]:
index = 0
# df_photos_am.loc[index,"id"]
df_photos_am.reset_index(drop=True).loc[0,'id']

'537318171761324'

In [ ]:
def get_photo(df_photos, index, access_token=access_token, filename=None,
    path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim\individual_photo_data'):
    """
    Get the url for the photo. This requires that the access token be active for the user of the photo.
    Parameters:
        - df_photos: DataFrame containing the API response.
        - index (int or list): Index in df_photos for which to obtain the url.
    Returns: URL for the photo.
    """
    df_photos.reset_index(drop=True, inplace=True)
    url_root = "https://graph.facebook.com/v15.0/"
    index_list = []
    url_list = []
    index_list.append(index)
    for index in index_list:
        request_url = f'{url_root}{str(df_photos.reset_index(drop=True).loc[index,"id"])}/picture?access_token='
        print('Request URL without access token:', request_url)
        request_url += access_token
        response = requests.get(request_url)
        print('Response status code: ',response.status_code)
        url_list.append(response.request.url)
    if filename:
        try:
            savepickle(url_list,filename+'_picture_url_'+str(index),'sav',json_path)
            savepickle(url_list,filename+'_api_response_'+str(index),'sav',json_path)
        except:
            print('Unable to save outputs')

    return url_list, response

In [ ]:
url_list, response = get_photo(df_photos, 0)

In [ ]:
url_list

['https://scontent.fcxh3-1.fna.fbcdn.net/v/t39.30808-6/325443079_709033990789318_5257377445815459026_n.jpg?stp=cp1_dst-jpg_p960x960&_nc_cat=110&ccb=1-7&_nc_sid=453a68&_nc_ohc=UhhHUM1TfuQAX95CI46&_nc_ht=scontent.fcxh3-1.fna&edm=AIv30VUEAAAA&oh=00_AfAzjFiZEmfvHgXEU29sj6ue-bcNRBP9rxz4JEsFkbYgtw&oe=63CB06AC']

In [ ]:
response

<Response [200]>

In [ ]:
url_list_am0, response_am0 = get_photo(df_photos_am, 0)

Request URL without access token: https://graph.facebook.com/v15.0/537318171761324/picture?access_token=
Response status code:  400


# Get BQ's posts

In [3]:
with open("credentials.json") as f:
    credentials = json.load(f)

bq_access_token = credentials['bq_access_token']
bq_user_id = credentials['bq_user_id']

In [5]:
df_bq, response_json_bq = get_user_post(bq_user_id, bq_access_token, pages=50, 
    filename='BQ_fb_posts_50page_2023-01-22')

Response status code:  400


KeyError: "Key 'data' not found. If specifying a record_path, all elements of data should have the path."

# Get Videos

In [ ]:
def get_user_media(user_id, access_token, pages=5, type='videos',filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):

    """
    Parmeters:
        - type ('videos' (default) or 'photos')
    """
    user_id = str(user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    url = f'{url_root}{user_id}/{type}?type=uploaded&access_token={access_token}'
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Request URL:', url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of photos:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df_videos_am, response_json_videos_am = get_user_media(am_user_id, access_token, pages=50,
    filename='AM_fb_videos_50page_2023-01-16')
df_videos_am

# SH 2023-01-16 16:01 Error message using Graph API explorer:
    # "(#10) This endpoint requires the 'pages_read_engagement' permission or the 'Page Public Content Access' feature. Refer to https://developers.facebook.com/docs/apps/review/login-permissions#manage-pages and https://developers.facebook.com/docs/apps/review/feature#reference-PAGES_ACCESS for details.",

# Get a Long-Lived User Access Token

In [ ]:
with open("credentials.json") as f:
    credentials = json.load(f)

access_token = credentials['access_token']
app_id = credentials['app_id']
app_secret = credentials['app_secret']

In [6]:
from datetime import datetime, timedelta
def prolong_access_token(credentials_json='credentials.json', access_token_key='access_token', 
    new_credentials_filename='credentials_long_lived.json'):
    
    """ SH 2023-01-17 23:04
    - Convert an access token to a Long-Lived User Access Token, which should last 60 days instead
    of 2 hours. 
    - Print the date and time of updated access_token expiry.
    - Create a new credentials JSON file with the Long-Lived User Access Token.

    Parameters:
        - credentials.json : JSON file containing the following fields:
            - 'access_token' or other key: User access token.
            - 'app_id'
            - 'app_secret'
        - access_token_key (str): Key to the relevant access_token in the 
            JSON file if different from 'access_token'
        - new_credentials_filename (str): Filename for saving the credentials file 
            with the long-lived user access token.
    Returns:
        - token_response: JSON object containing API GET response.

    Relevant API documentation:
    https://developers.facebook.com/docs/facebook-login/guides/access-tokens/get-long-lived
    """
    # Retrieve credentials
    with open("credentials.json") as f:
        credentials = json.load(f)
    access_token = credentials[access_token_key]
    app_id = credentials['app_id']
    app_secret = credentials['app_secret']

    # Make API request: Query the GET oauth/access_token endpoint
    url_root = "https://graph.facebook.com/v15.0/oauth/access_token"
    request_url = f'{url_root}?grant_type=fb_exchange_token&client_id={app_id}&client_secret={app_secret}&fb_exchange_token={access_token}'
    response = requests.get(request_url)
    print('Response status code: ',response.status_code)
    try:
        response_json = response.json()
        response_json['request_url'] = request_url
        try:
            new_access_token = response_json['access_token']
            credentials[access_token_key] = new_access_token
            time_to_expiry = timedelta(seconds=response_json['expires_in'])
            now = datetime.now()
            credentials['token_expiry'] = (datetime.now() + time_to_expiry).strftime("%Y-%m-%d %H:%M")
            print('Updated token expiry:', credentials['token_expiry'])
            with open(new_credentials_filename,'w') as json_file:
                json.dump(credentials, json_file)
                print('New credentials file created:', new_credentials_filename)
        except:
            print('Unable to save new credentials; check request response')
        return response_json
    except:
        print('Unable to get response JSON; check request response')
        return response


In [159]:

token_response = prolong_access_token()

Response status code:  200
Updated token expiry: 2023-03-18 21:09
New credentials file created: credentials_long_lived.json


In [ ]:
# Check that the new credentials file loads properly
new_credentials_filename='credentials_long_lived.json'

with open(new_credentials_filename) as json_file:
    credentials2 = json.load(json_file)
credentials2

In [7]:
token_response = prolong_access_token(access_token_key='bq_access_token')

Response status code:  400
Unable to save new credentials; check request response


# Get group feed

In [ ]:
token_response = prolong_access_token(access_token_key='am_access_token')

In [ ]:
# endpoint for Strong and Sassy group
group_feed_url = "https://graph.facebook.com/v15.0/2139238999669147/feed?access_token="+access_token

In [ ]:
def get_group_feed(credentials_json='credentials_long_lived.json', group_id_key='SSC_group_id', 
    access_token_key='am_access_token',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """
    Parameters:
        - credentials.json : JSON file containing the required credentials:
            - group_id for the Facebook group.
            - access_token for the user.
        - group_id_key and access_token_key (str): Keys to the relevant group_id and access_token in the 
            JSON file.
    Returns:
        - 
    """
    # Retrieve credentials
    with open("credentials.json") as f:
        credentials = json.load(f)
    access_token = credentials[access_token_key]
    group_id = credentials[group_id_key]

    # Make API request:
    url_root = "https://graph.facebook.com/v15.0/"
    url = f'{url_root}{group_id}/feed?access_token={access_token}'


# Process the data

In [164]:
df_50p.head()

,created_time,message,id
0,2023-01-04T15:06:57+0000,Charity event in Roscommon - Saturday 7th Janu...,545505737609234_537318481761293
1,2022-11-29T18:55:49+0000,Nutrition and Training Workshop 💪\n\nMy ethos ...,545505737609234_509802061179602
2,2022-10-04T22:39:50+0000,‘Why your metabolism is not broken’\n\nReally ...,545505737609234_467470882079387
3,2022-09-18T20:34:20+0000,Anyone else on TikTok? 🎯,545505737609234_455027709990371
4,2022-09-18T06:19:36+0000,Completed it mate ✅,545505737609234_454597556700053


In [177]:
df_50p.iloc[-8:-3]

,created_time,message,id
45,2021-06-07T10:48:27+0000,https://www.facebook.com/100000504382136/posts...,545505737609234_150885450404600
46,2021-05-11T05:30:40+0000,Aoibheann's Experience:\n.\n.\nI started the p...,545505737609234_133207925505686
47,2021-05-06T04:43:38+0000,If you could eat one food and still achieve yo...,545505737609234_129813195845159
48,2021-05-03T00:12:56+0000,‼️I'm looking for 5 ladies who want to become ...,545505737609234_128188466007632
49,2021-04-27T04:45:39+0000,Snack Options ✅\n.\n.\nA question I get so fre...,545505737609234_126493706177108


In [269]:
df_50p.iloc[-8:-3]['message']

45    https://www.facebook.com/100000504382136/posts...
46    Aoibheann's Experience:\n.\n.\nI started the p...
47    If you could eat one food and still achieve yo...
48    ‼️I'm looking for 5 ladies who want to become ...
49    Snack Options ✅\n.\n.\nA question I get so fre...
Name: message, dtype: object

Options for feature transformation:
* bag of words / CountVectorizer
* vectorization using pre-trained model

Given the small amount of data and the lack of labels (i.e. no business metrics for this time period), it doesn't make to:
* Perform topic modelling
* Perform any modelling

For now, just do EDA; look at bag of words because word embeddings cannot be interpreted by humans

In [47]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
def preprocess_post_text(docs):
    """
    Prepare data from text documents for NLP:
    - Remove all the special characters
    - Remove all single characters
    - Substitute multiple spaces with single space
    - Convert to lowercase

    Parameters:
    docs (n x 1 array or string): Documents.

    Returns: Array of processed docs.
    """
    clean_docs = []
    wnl = WordNetLemmatizer()
    for doc in docs:
        try:
            # Split text into single words (also gets rid of extra white spaces)
            words = word_tokenize(doc)

            # Convert to lower case
            words = [word.lower() for word in words]

            # Lemmatize words (must be done after conversion to lower case)
            words = [wnl.lemmatize(word) for word in words]

            # Remove all single characters
            words = [word for word in words if len(word) > 2]
            
            # join words back together as a string
            words = ''.join([word+' ' for word in words])

            # Remove any URLs 
            words = re.sub(r'\w*\.+\w*', '', words) # Remove periods in middle of word
            words = re.sub(r'\w*/+\w*', '', words) # remove forward slash in middle of word
            words = re.sub(r'\w*/+\w*', '', words)

            # remove special characters
            words = ''.join([char for char in words if char not in string.punctuation])
            clean_docs.append(words)
        except: # In case value is nan
            clean_docs.append(doc)
        
    return np.array(clean_docs)

from sklearn.feature_extraction.text import CountVectorizer
from custom_nlp import *
def post_eda(df):
    df = process_timestamp(df)
    df['text'] = preprocess_post_text(df['text'])
    vect = CountVectorizer()
    vect.fit(df['text'])
    vector = vect.transform(df['text'])
    print('shape: ', vector.shape)
    vector_df = pd.DataFrame(vector.toarray(), columns=vect.get_feature_names_out())

    return df, vect, vector_df

df, vector, vector_df = post_eda(df_50p.iloc[-8:-3])

shape:  (5, 214)


In [48]:
df

,timestamp,text,date,year,month,day_of_week,time,hour
0,2021-06-07 10:48:27+00:00,http who want make change like this 👇👇👇,2021-06-07,2021,6,0,10:48:27,10
1,2021-05-11 05:30:40+00:00,aoibheann experience started the programme wan...,2021-05-11,2021,5,1,05:30:40,5
2,2021-05-06 04:43:38+00:00,you could eat one food and still achieve your ...,2021-05-06,2021,5,3,04:43:38,4
3,2021-05-03 00:12:56+00:00,‼️i looking for lady who want become strong an...,2021-05-03,2021,5,0,00:12:56,0
4,2021-04-27 04:45:39+00:00,snack option question get frequently and area ...,2021-04-27,2021,4,1,04:45:39,4


In [49]:
vector_df

,2021,able,achieve,actually,advocate,after,all,also,amanda,amazing,...,without,woman,work,working,would,wouldn,year,you,your,yourself
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,1,0,3,0,...,0,0,0,1,1,1,1,1,1,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
3,0,1,0,2,1,0,0,1,0,1,...,0,4,1,0,0,0,0,7,2,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


# *End of Page*